In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [119]:
import numpy as np

In [99]:
# Cell 2: Load and Check Data
df = pd.read_excel('/content/drive/MyDrive/TechTalentAssignment/Data & Topics.xlsx')
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df


,Context,Training Topic,Predicted Topic
0,Wiping out graft to fatten coffers Will the ...,Corruption,NaN
1,"দূর্নীতির আখড়া গোপালগঞ্জ হাসপাতাল,ধরলো দুদক ...",Corruption,NaN
2,মুজিব কিল্লা নির্মাণে অর্থ তছরুপে জড়িতদের বিরু...,Corruption,NaN
3,অভিযোগের বিষয়ে জানতে গেলে বোবার অভিনয়... বিস্...,Corruption,NaN
4,"ব্রিজ নির্মাণ হলেও দেয়া হয়নি এপ্রোচ সড়ক, গোপাল...",Corruption,NaN
...,...,...,...
584,"Since August 5, the rule of darkness called #M...",Mob Justice,NaN
585,ধানমন্ডি ৩২ এত দিন টিকে থাকাটাই ছিল সরকারের ব...,Mob Justice,NaN
586,"♦মব জাস্টিসের আতঙ্কে পুলিশ ও জনগণ, আইন-শৃঙ্খলা...",Mob Justice,NaN
587,"In #Dhaka today, 2 #foreigncitizens faced the ...",Mob Justice,NaN


In [27]:
# Cell 3: Install Required Packages
!pip install langdetect
# !pip install --upgrade --force-reinstall numpy
# !pip install --upgrade --force-reinstall scipy
!pip install nltk
!pip install bltk

  Using cached bltk-1.2.tar.gz (17.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [28]:
!pip install sbnltk

  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bnlp-toolkit 4.0.3 requires scipy==1.10.1, but you have scipy 1.15.2 which is incompatible.


In [29]:
# Cell 4: Import Text Processing Libraries
import pandas as pd
import re
import string
from langdetect import detect
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sbnltk.Stemmer import stemmerOP
from sbnltk.Preprocessor import preprocessor
from sbnltk.Tokenizer import wordTokenizer, sentenceTokenizer

In [87]:
from textblob import TextBlob

In [30]:
# Cell 5: Download NLTK Resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [98]:
# Cell 6: Define Text Processing Functions

# Emoji removal pattern
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # Emoticons
    u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & Map
    u"\U0001F1E0-\U0001F1FF"  # Flags
    u"\U00002700-\U000027BF"  # Dingbats
    u"\U0001F900-\U0001F9FF"  # Supplemental symbols
    u"\U0001FA70-\U0001FAFF"  # Extended-A (🫵)
    u"\U00002600-\U000026FF"  # Misc symbols
    u"\U0001F000-\U0001F02F"  # Mahjong
    u"\U0000203C"             # Double exclamation
    u"\uFE0F"                 # Variation Selector-16
    "]+",
    flags=re.UNICODE
)


# Custom punctuation (Bangla, Urdu, Arabic)
custom_punct = '।॥''""…،؛؟'
all_punct = string.punctuation + custom_punct

# Load stopwords
en_stop = set(stopwords.words('english'))

# Initialize processors
en_lem = WordNetLemmatizer()
bn_preprocessor = preprocessor()
bn_word_tokenizer = wordTokenizer()
# bn_sent_tokenizer = sentenceTokenizer()
bn_stemmer = stemmerOP()

# Detect Bengali characters
def is_bengali(word):
    return bool(re.search(r'[\u0980-\u09FF]', word))

# Clean and preprocess text
def preprocess_text(text):
    text = str(text)

    # Remove URLs, hashtags, mentions
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"@\w+", "", text)

    # Remove emojis
    text = emoji_pattern.sub('', text)

    # Remove punctuations
    text = re.sub(f"[{re.escape(all_punct)}]", "", text)

    # Normalize space
    text = re.sub(r'\s+', ' ', text).strip()

    try:
        lang = detect(text)
    except:
        lang = 'en'

    # For Bengali text, use SBNLTK tokenizer
    if (lang == 'bn'):
        try:
            # print(text)
            # # Preprocess Bengali text
            # text = bn_preprocessor.dust_removal(text)
            text = bn_preprocessor.word_normalize(text)
            # Tokenize into words
            tokens = bn_word_tokenizer.basic_tokenizer(text)

            # Apply stemming to each token
            tokens = [bn_stemmer.stemWord(t) for t in tokens]
            # Apply dust remove to each token
            tokens = [bn_preprocessor.dust_removal(t) for t in tokens]
            # print(tokens)
        except:
            # Fallback to NLTK if SBNLTK fails
            tokens = word_tokenize(text)
            tokens = [t.lower() for t in tokens if t.isalpha()]
    else:
        # For English, use NLTK
        textBlb = TextBlob(text)
        textBlb = textBlb.correct()
        text = textBlb.string
        # Tokenize into words
        tokens = word_tokenize(text)
        tokens = [t.lower() for t in tokens if t.isalpha()]
        print(tokens)

    clean_tokens = []
    for token in tokens:
        if is_bengali(token):
            # Bengali tokens are already preprocessed and stemmed
            clean_tokens.append(token)
        else:
            if token not in en_stop:
                lemma = en_lem.lemmatize(token)
                clean_tokens.append(lemma)

    return clean_tokens


In [100]:
# Cell 7: Apply Preprocessing to Data
df['clean_context'] = df['Context'].apply(preprocess_text)

['wiping', 'out', 'graft', 'to', 'fatten', 'offers', 'will', 'the', 'government', 's', 'proposal', 'under', 'budget', 'to', 'eradicate', 'corruption', 'and', 'improve', 'management', 'of', 'the', 'nation', 's', 'finances', 'help', 'to', 'raise', 'revenue']
['bangladesh', 'interim', 'governments', 'chief', 'adviser', 'or', 'muhammad', 'guns', 'has', 'assured', 'british', 'parliamentarian', 'up', 'but', 'that', 'the', 'next', 'general', 'election', 'will', 'be', 'completely', 'free', 'and', 'fair', 'head', 'more']
['in', 'a', 'time', 'of', 'global', 'is', 'working', 'to', 'stop', 'diseases', 'driven', 'by', 'environmental', 'degradation', 'ensure', 'human', 'animal', 'health', 'more', 'on', 'our', 'approach']
['with', 'usaid', 's', 'support', 'the', 'international', 'salon', 'of', 'saharan', 'and', 'basin', 'tourism', 'brought', 'together', 'local', 'suppliers', 'and', 'buyers', 'for', 'oneonone', 'meetings', 'showcasing', 'this', 'region', 's', 'tapped', 'beauty', 'and', 'potential', 'h

In [104]:
df


,Context,Training Topic,Predicted Topic,clean_context
0,Wiping out graft to fatten coffers Will the ...,Corruption,NaN,"[wiping, graft, fatten, offer, government, pro..."
1,"দূর্নীতির আখড়া গোপালগঞ্জ হাসপাতাল,ধরলো দুদক ...",Corruption,NaN,"[দুর্নিতির, আখড়া, গোপালগঞ্জ, হাসপাতালধরলো, দুদক]"
2,মুজিব কিল্লা নির্মাণে অর্থ তছরুপে জড়িতদের বিরু...,Corruption,NaN,"[মুজিব, কিল্লা, নির্মানে, অর্থ, তছরুপ, জড়িত, ব..."
3,অভিযোগের বিষয়ে জানতে গেলে বোবার অভিনয়... বিস্...,Corruption,NaN,"[অভিযোগ, বিষ, জান, গ, বোব, অভিন, বিস্তারিত, কম..."
4,"ব্রিজ নির্মাণ হলেও দেয়া হয়নি এপ্রোচ সড়ক, গোপাল...",Corruption,NaN,"[ব্রিজ, নির্মান, হল, দেয়া, হয়নি, এপ্রোচ, সড়ক, ..."
...,...,...,...,...
584,"Since August 5, the rule of darkness called #M...",Mob Justice,NaN,"[since, august, rule, darkness, called, going,..."
585,ধানমন্ডি ৩২ এত দিন টিকে থাকাটাই ছিল সরকারের ব...,Mob Justice,NaN,"[ধানমন্ডি, ৩২, এত, দিন, টি, থাকাটাই, ছিল, সরকা..."
586,"♦মব জাস্টিসের আতঙ্কে পুলিশ ও জনগণ, আইন-শৃঙ্খলা...",Mob Justice,NaN,"[মব, জাস্টিস, আতঙ্ক, পুলিশ, ও, জন, আইনশৃঙ্খলা,..."
587,"In #Dhaka today, 2 #foreigncitizens faced the ...",Mob Justice,NaN,"[today, faced, part, amp, parcel, august]"


No charts were generated by quickchart


In [110]:
df = df[df['clean_context'].map(lambda x: len(x) > 0)]



In [111]:
df['clean_context']

,clean_context
0,"[wiping, graft, fatten, offer, government, pro..."
1,"[দুর্নিতির, আখড়া, গোপালগঞ্জ, হাসপাতালধরলো, দুদক]"
2,"[মুজিব, কিল্লা, নির্মানে, অর্থ, তছরুপ, জড়িত, ব..."
3,"[অভিযোগ, বিষ, জান, গ, বোব, অভিন, বিস্তারিত, কম..."
4,"[ব্রিজ, নির্মান, হল, দেয়া, হয়নি, এপ্রোচ, সড়ক, ..."
...,...
584,"[since, august, rule, darkness, called, going,..."
585,"[ধানমন্ডি, ৩২, এত, দিন, টি, থাকাটাই, ছিল, সরকা..."
586,"[মব, জাস্টিস, আতঙ্ক, পুলিশ, ও, জন, আইনশৃঙ্খলা,..."
587,"[today, faced, part, amp, parcel, august]"


In [54]:
bn_stemmer.stemWord('দূর্নীতির আখড়া গোপালগঞ্জ হাসপাতাল,ধরলো দুদক')

'দূর্নীতির আখড়া গোপালগঞ্জ হাসপাতাল,ধরলো দুদক'

In [128]:
X = df['clean_context']
y = df['Training Topic']

In [129]:
y.nunique()

20

In [130]:
np.unique(y)

array(['Corruption', 'Culture & Lifestyle', 'Diplomacy', 'Education',
       'Election', 'Environment', 'Governance & Policy Reform',
       'International affairs', 'Islamic Fundamentalism', 'Law and Order',
       'Mob Justice', 'National Defence', 'Natural Disaster', 'Politics',
       'Religion', 'Religious Conflict', 'Sports', 'Terrorism',
       'Trade & Commodity Price', 'Women Rights'], dtype=object)

In [133]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# First, let's check the class distribution to handle the stratification issue
print("Checking class distribution...")
class_counts = y.value_counts()
print(class_counts)

# Filter out classes with only one sample
print("\nRemoving classes with only one sample...")
valid_classes = class_counts[class_counts >= 2].index
filtered_df = df[df['Training Topic'].isin(valid_classes)]
print(f"Original dataset size: {len(df)}")
print(f"Filtered dataset size: {len(filtered_df)}")

# Get the new X and y from filtered dataset
X = filtered_df['clean_context']
y = filtered_df['Training Topic']

# Now perform the train-test split without error
print("\nPerforming train-test split...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model on resampled data
print("\nTraining logistic regression model on resampled data...")
model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    multi_class='multinomial',
    solver='lbfgs',
    random_state=42
)
model.fit(X_train_resampled, y_train_resampled)

# Make predictions
print("Making predictions...")
y_train_pred = model.predict(X_train_tfidf)  # Predict on original training data
y_test_pred = model.predict(X_test_tfidf)

# Calculate metrics
print("\n--- Training set evaluation (on original training data) ---")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred))

print("\n--- Test set evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Generate confusion matrix for training set
print("\nGenerating confusion matrices...")

# Plot normalized confusion matrix for test set (most important)
plt.figure(figsize=(12, 10))
cm_test = confusion_matrix(y_test, y_test_pred)
cm_test_normalized = cm_test.astype('float') / cm_test.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_test_normalized, annot=True, fmt='.2f', cmap='Greens',
            xticklabels=model.classes_, yticklabels=model.classes_)
plt.title('Normalized Confusion Matrix (Test Set)')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Save the model and vectorizer
print("\nSaving model and vectorizer...")
import joblib
joblib.dump(model, 'text_classification_model_resampled.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
print("Model and vectorizer saved successfully!")

# Identify the most important features for each class
print("\nMost important features for each class:")
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame to store feature importance for visualization
feature_importance_data = []

for i, class_label in enumerate(model.classes_):
    # Get top features with highest coefficients for this class
    top_features_idx = np.argsort(model.coef_[i])[-10:]
    top_features = [feature_names[idx] for idx in top_features_idx]
    top_coefficients = [model.coef_[i][idx] for idx in top_features_idx]

    # Add to the feature importance data
    for feature, coef in zip(top_features, top_coefficients):
        feature_importance_data.append({
            'Class': class_label,
            'Feature': feature,
            'Importance': coef
        })

    # Print top features for this class
    print(f"Class '{class_label}': {', '.join(top_features)}")

# Create a bar chart showing performance metrics per class
print("\nVisualizing per-class performance...")
class_report = classification_report(y_test, y_test_pred, output_dict=True)
class_df = pd.DataFrame(class_report).transpose()
metrics_df = class_df[['precision', 'recall', 'f1-score']].iloc[:-3]  # Remove avg/total rows

plt.figure(figsize=(12, 8))
metrics_df.plot(kind='bar', figsize=(12, 8))
plt.title('Performance Metrics by Class')
plt.xticks(rotation=90)
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Display detailed per-class metrics
print("\nDetailed per-class metrics (sorted by F1-score):")
print(class_df[['precision', 'recall', 'f1-score', 'support']].sort_values(by='f1-score'))

# Display class distribution comparison
print("\nClass counts comparison (original train vs. resampled train vs. test):")
original_train_counts = pd.Series(y_train).value_counts().sort_index()
resampled_train_counts = pd.Series(y_train_resampled).value_counts().sort_index()
test_counts = pd.Series(y_test).value_counts().sort_index()

comparison_df = pd.DataFrame({
    'Original Train': original_train_counts,
    'Resampled Train': resampled_train_counts,
    'Test': test_counts
}).fillna(0).astype(int)

print(comparison_df)

# Save the model and vectorizer
print("\nSaving model and vectorizer...")
import joblib
joblib.dump(model, 'text_classification_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
print("Model and vectorizer saved successfully!")

# Identify the most important features for each class
print("\nMost important features for each class:")
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame to store feature importance for visualization
feature_importance_data = []

for i, class_label in enumerate(model.classes_):
    # Get top features with highest coefficients for this class
    top_features_idx = np.argsort(model.coef_[i])[-15:]
    top_features = [feature_names[idx] for idx in top_features_idx]
    top_coefficients = [model.coef_[i][idx] for idx in top_features_idx]

    # Add to the feature importance data
    for feature, coef in zip(top_features, top_coefficients):
        feature_importance_data.append({
            'Class': class_label,
            'Feature': feature,
            'Importance': coef
        })

    # Print top features for this class
    print(f"Class '{class_label}': {', '.join(top_features)}")

# Visualize top features for the most prevalent classes
top_classes = class_counts.index[:5]  # Top 5 classes
plt.figure(figsize=(15, 10))
for i, class_name in enumerate(top_classes):
    class_data = [item for item in feature_importance_data if item['Class'] == class_name]
    class_data = sorted(class_data, key=lambda x: x['Importance'], reverse=True)[:10]

    plt.subplot(2, 3, i+1)
    features = [item['Feature'] for item in class_data]
    importances = [item['Importance'] for item in class_data]

    y_pos = np.arange(len(features))
    plt.barh(y_pos, importances, align='center')
    plt.yticks(y_pos, features)
    plt.xlabel('Coefficient Value')
    plt.title(f"Top Features: {class_name}")

plt.tight_layout()
plt.show()

# Performance per class
print("\nDetailed per-class metrics (sorted by F1-score):")
class_report = classification_report(y_test, y_test_pred, output_dict=True)
class_df = pd.DataFrame(class_report).transpose()
print(class_df[['precision', 'recall', 'f1-score', 'support']].sort_values(by='f1-score'))

Checking class distribution...
Training Topic
Mob Justice                   124
Law and Order                  66
Politics                       59
Islamic Fundamentalism         45
International affairs          34
Religion                       33
Corruption                     30
National Defence               27
Diplomacy                      24
Governance & Policy Reform     20
Women Rights                   19
Sports                         15
Terrorism                      12
Election                       11
Culture & Lifestyle            10
Trade & Commodity Price         6
Education                       4
Environment                     2
Natural Disaster                2
Name: count, dtype: int64

Removing classes with only one sample...
Original dataset size: 544
Filtered dataset size: 543

Performing train-test split...

Training logistic regression model on resampled data...


NameError: name 'X_train_resampled' is not defined

NameError: name 'X_train' is not defined